# Procedure when data added ; Start from "pre_inbody." [HERE]

## Can use "[RESET/RESTART]" because recalculated data is stored in pickle
1. Move new data from "C:\Users\bhuns\OneDrive\___Health Data\__DD studies\InBody CSV\ib97" into "JL_1/data/ib97"
2. GO to "pre_inbody"  and Hit >> [RESET/RESTART] to erase old results, reset kernal and run the cell. [Results in pickle]
3. GO to "nrmlz_data_dict"  and Hit >> [RESET/RESTART] to erase old results, reset kernal and run the cell. [Results in pickle]
4. GO to "plot_sandbox"  and Hit >> [RESET/RESTART] to erase old results, reset kernal and run the cell. [Results in graphs]
5. 

In [365]:
import sys
print(sys.executable)
print("note: THIS IS THE DIRECTORY PYTHON IS WORKING IN.")



/home/bhuns/miniconda3/bin/python
note: THIS IS THE DIRECTORY PYTHON IS WORKING IN.


# Import

In [366]:
# Imports required for Loading, sorting .csx files to create specific data sets ie mrn inbody readings. 
%run ./sys_funcs.py              # loads all the def functions in sys_funcs.py into memory
#import sys_funcs                 # gives access to these def function digitalform that are in memory
from pathlib import Path
import csv
import pandas as pd
import numpy as np
import tkinter as tk
import pickle
from pathlib import Path
import csv
import os
import sys
from datetime import datetime
from datetime import time
from sys_funcs import read_csv_to_array
from sys_funcs import clean_wsl_path
from sys_funcs import array_to_dt_row_dict
from sys_funcs import make_blnk_update_row_dict
from sys_funcs import transpose_csv_to_col_dict
#from sys_funcs import update_values_with_config, get_update_result
from sys_funcs import transfer_updates
from sys_funcs import get_dtv_range
from sys_funcs import universal_import
from sys_funcs import parse_inbody_timestamp
from sys_funcs import build_lut
from sys_funcs import extract_a_column_as_df
from sys_funcs import extract_multicolumns_as_df
from sys_funcs import validate_and_sort_timestamps
from sys_funcs import extract_and_filter_by_time_window
from sys_funcs import read_file_dual_path
from sys_funcs import write_file_dual_path
from sys_funcs import asc_to_csv_cnv
#from sys_funcs import 

# DEF functions for this workbook

In [367]:
# name the list in the call and this function will:
# ask for a substring and make a list of names that contain it.
def filter_column_names_interactive(df_col_nms):
    """
    Prompts for a substring and prints matching column names from df_col_nms.
    Assumes df_col_nms is a single-column DataFrame of column names.
    """
    substring = input("Enter substring to filter column names: ").strip()
    matches = df_col_nms[df_col_nms.iloc[:, 0].str.contains(substring, case=False, na=False)]
    
    if matches.empty:
        print(f"No column names contain '{substring}'.")
    else:
        print(f"Column names containing '{substring}':")
        for name in matches.iloc[:, 0]:
            print(f"  - {name}")


In [368]:
# this def function takes raw imported a data_dict cleans up the '14. Test Date / Time' then`
# attachs a "Cleaned_Timestamp" and a "dtv" 
import numpy as np
import pandas as pd
from datetime import datetime

def clean_data_dict(data_dict):
    """
    Cleans the input data_dict by:
    - Stripping and truncating '14. Test Date / Time' to 14 characters
    - Creating a 'Cleaned_Timestamp' column
    - Creating a 'dtv' column: days since 1/1/1900
    Returns a cleaned copy of the input dictionary.
    """
    cleaned_dict = data_dict.copy()

    # Step 1: Clean and truncate timestamp strings
    cleaned_dict["Cleaned_Timestamp"] = [
        str(ts).strip()[:14] if str(ts).strip().lower() != "nan" and len(str(ts).strip()) >= 14 else np.nan
        for ts in cleaned_dict.get("14. Test Date / Time", [])
    ]

    # Step 2: Convert to datetime and calculate days since 1/1/1900
    base_date = datetime(1900, 1, 1)
    def safe_parse(ts):
        try:
            return (datetime.strptime(ts, "%Y%m%d%H%M%S") - base_date).days
        except Exception:
            return np.nan

    cleaned_dict["dtv"] = [
        safe_parse(ts) if isinstance(ts, str) and len(ts) == 14 else np.nan
        for ts in cleaned_dict["Cleaned_Timestamp"]
    ]

    return cleaned_dict


# Create **ib970** by Loading **970.csv** files

In [369]:
ib970_a = universal_import(
    folder_path="/home/bhuns/JL_2/data/ib97",
    pattern="091725*",
    df_name="ib970_a"
)

ib970_b = universal_import(
    folder_path="/home/bhuns/JL_2/data/ib97",
    pattern="251201*",
    df_name="ib970_b"
)

# Combine them (assuming they return DataFrames)
import pandas as pd
ib970 = pd.concat([ib970_a, ib970_b], ignore_index=True)



✅ Loaded 091725-1_20251118042051.csv with ISO-8859-1
✅ Loaded 091725-1_20251023065307.csv with ISO-8859-1
✅ Loaded 091725-1_20250928235640.csv with ISO-8859-1
✅ Loaded 091725-1_20251007213459 - Copy.csv with ISO-8859-1
✅ Loaded 091725-1_20251006232452 - Copy.csv with ISO-8859-1
✅ Loaded 091725-1_20251025233431.csv with ISO-8859-1
✅ Loaded 091725-1_20251005220517 - Copy.csv with ISO-8859-1
✅ Loaded 091725-1_20251124075021.csv with ISO-8859-1
✅ Loaded 091725-1_20251021000847.csv with ISO-8859-1
✅ Loaded 091725-1_20251101213555.csv with ISO-8859-1
✅ Loaded 091725-1_20251116230333.csv with ISO-8859-1
✅ Loaded 091725-1_20251130055027.csv with ISO-8859-1
✅ Loaded 091725-1_20251019230750.csv with ISO-8859-1
✅ Loaded 091725-1_20251017091340.csv with ISO-8859-1
✅ Loaded 091725-1_20251002065435.csv with ISO-8859-1
✅ Loaded 091725-1_20251019074606.csv with ISO-8859-1
✅ Loaded 091725-1_20251118230931.csv with ISO-8859-1
✅ Loaded 091725-1_20251108074302.csv with ISO-8859-1
✅ Loaded 091725-1_2025092

# Create **ib970cln** *and* **df_ib970cln_col_nms** from **ib970**

## Cleanup the **'14. Test Date / Time'** so **datetime** will work and add cols ie. keys:
1. **Cleaned_Timestamp**
2. **dtv**

In [370]:
from collections.abc import Mapping
import pandas as pd

def kind_of(var):
    if isinstance(var, pd.DataFrame):
        return "dataframe"
    if isinstance(var, Mapping):   # covers dict, OrderedDict, defaultdict, etc.
        return "dict-like"
    return "other"

# Usage

print(kind_of(pd.DataFrame({"ib970cln ":[1,2]})))  # dataframe
print(kind_of({"ib970cln ": 1}))                   # dict-like
print(kind_of([1,2,3]))                    # other


dataframe
dict-like
other


In [371]:
kind_of(ib970)

'dataframe'

In [372]:
# this calls the def clean_data_dict () to clean bad testdate data and add "dtv" & "Cleaned_Timestamp" NANs still included
ib970cln = clean_data_dict(ib970)
print("ib970cln has been created with dtv & Cleaned_Timestamp cols added ")

ib970cln has been created with dtv & Cleaned_Timestamp cols added 


In [373]:
# verify still not in temporal order yet
#
ib970cln       #this has all the rows but not in chronological oreder

,1. Name,2. ID,3. Height,4. Date of Birth,5. Gender,6. Age,7. Mobile Number,8. Phone Number,9. Zip Code,10. Address,...,246. R/Ht,247. Xc/Ht,248. HGS of Left Arm 1st,249. HGS of Left Arm 2nd,250. HGS of Right Arm 1st,251. HGS of Right Arm 2nd,252. HGS/WT,Unnamed: 252,Cleaned_Timestamp,dtv
0,bjh970,091725-1,6ft 06.0in,1939.02.23.,M,86.0,-,-,-,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20251118042051,45977.0
1,bjh970,091725-1,6ft 06.0in,1939.02.23.,M,86.0,-,-,-,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20251023065307,45951.0
2,bjh970,091725-1,6ft 06.0in,1939.02.23.,M,86.0,-,-,-,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20250928235640,45926.0
3,bjh970,091725-1,6ft 06.0in,1939.02.23.,M,86.0,-,-,-,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20251007213459,45935.0
4,bjh970,091725-1,6ft 06.0in,1939.02.23.,M,86.0,-,-,-,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20251006232452,45934.0
5,bjh970,091725-1,6ft 06.0in,1939.02.23.,M,86.0,-,-,-,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20251025233431,45953.0
6,bjh970,091725-1,6ft 06.0in,1939.02.23.,M,86.0,-,-,-,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20251005220517,45933.0
7,bjh970,091725-1,6ft 06.0in,1939.02.23.,M,86.0,-,-,-,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20251124075021,45983.0
8,bjh970,091725-1,6ft 06.0in,1939.02.23.,M,86.0,-,-,-,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20251021000847,45949.0
9,bjh970,091725-1,6ft 06.0in,1939.02.23.,M,86.0,-,-,-,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20251101213555,45960.0


### Read the **df_ib970cln_col_nms** *from* **ib970cln**

In [374]:
df_ib970cln_col_nms = pd.DataFrame(ib970cln.columns)
print("df_ib970cln_col_nms has been created ")
# df_ib970cln_col_nms

df_ib970cln_col_nms has been created 


In [375]:
# verify
# 
df_ib970cln_col_nms

,0
0,1. Name
1,2. ID
2,3. Height
3,4. Date of Birth
4,5. Gender
5,6. Age
6,7. Mobile Number
7,8. Phone Number
8,9. Zip Code
9,10. Address


## Save **ib970cln** and **df_ib970cln_col_nms** to **pickle**

In [376]:
# Save ib970cln to a file
import pickle
with open("ib970cln.pkl", "wb") as f:
    pickle.dump(ib970cln, f)
print(" ib970cln is saved to pickle")


 ib970cln is saved to pickle


In [377]:
# verify 
# 
ib970cln    # all rows but not in chronological order

,1. Name,2. ID,3. Height,4. Date of Birth,5. Gender,6. Age,7. Mobile Number,8. Phone Number,9. Zip Code,10. Address,...,246. R/Ht,247. Xc/Ht,248. HGS of Left Arm 1st,249. HGS of Left Arm 2nd,250. HGS of Right Arm 1st,251. HGS of Right Arm 2nd,252. HGS/WT,Unnamed: 252,Cleaned_Timestamp,dtv
0,bjh970,091725-1,6ft 06.0in,1939.02.23.,M,86.0,-,-,-,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20251118042051,45977.0
1,bjh970,091725-1,6ft 06.0in,1939.02.23.,M,86.0,-,-,-,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20251023065307,45951.0
2,bjh970,091725-1,6ft 06.0in,1939.02.23.,M,86.0,-,-,-,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20250928235640,45926.0
3,bjh970,091725-1,6ft 06.0in,1939.02.23.,M,86.0,-,-,-,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20251007213459,45935.0
4,bjh970,091725-1,6ft 06.0in,1939.02.23.,M,86.0,-,-,-,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20251006232452,45934.0
5,bjh970,091725-1,6ft 06.0in,1939.02.23.,M,86.0,-,-,-,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20251025233431,45953.0
6,bjh970,091725-1,6ft 06.0in,1939.02.23.,M,86.0,-,-,-,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20251005220517,45933.0
7,bjh970,091725-1,6ft 06.0in,1939.02.23.,M,86.0,-,-,-,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20251124075021,45983.0
8,bjh970,091725-1,6ft 06.0in,1939.02.23.,M,86.0,-,-,-,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20251021000847,45949.0
9,bjh970,091725-1,6ft 06.0in,1939.02.23.,M,86.0,-,-,-,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20251101213555,45960.0


In [378]:
# verify 
# 
ib970cln_dtstmp = ib970cln[["dtv", "Cleaned_Timestamp"]]    # time and datestamp df

In [379]:
# verify 
# 
ib970cln_dtstmp    #["ib970cln_dtstmp]

,dtv,Cleaned_Timestamp
0,45977.0,20251118042051
1,45951.0,20251023065307
2,45926.0,20250928235640
3,45935.0,20251007213459
4,45934.0,20251006232452
5,45953.0,20251025233431
6,45933.0,20251005220517
7,45983.0,20251124075021
8,45949.0,20251021000847
9,45960.0,20251101213555


In [380]:
# Save df_ib970cln_col_nms to a file
import pickle
with open("df_ib970cln_col_nms.pkl", "wb") as f:
    pickle.dump( df_ib970cln_col_nms, f)
print("  df_ib970cln_col_nms is saved to pickle")

  df_ib970cln_col_nms is saved to pickle


In [381]:
# Save ib970cln_dtstmp to a file
import pickle
with open("ib970cln_dtstmp.pkl", "wb") as f:
    pickle.dump( ib970cln_dtstmp, f)
print("  ib970cln_dtstmp is saved to pickle")

  ib970cln_dtstmp is saved to pickle


In [382]:
# verify
# ib970mrn_grp1_nms

In [383]:
# verify
# 
ib970cln_dtstmp

,dtv,Cleaned_Timestamp
0,45977.0,20251118042051
1,45951.0,20251023065307
2,45926.0,20250928235640
3,45935.0,20251007213459
4,45934.0,20251006232452
5,45953.0,20251025233431
6,45933.0,20251005220517
7,45983.0,20251124075021
8,45949.0,20251021000847
9,45960.0,20251101213555


# Creating  **ib970mrn_grp_dict** from **df_ib970cln_col_nms**

##  **ib970mrn_grp1** from **ib970cln**

### Procedure

#### 
1. used to specify the keys[data_cols] needed in **grp1**
2. The are selected from **df_ib970cln_col_nms**
3. They are chosen manually by copying the names from **df_ib970cln_col_nms list** and pasting them into **ib970mrn_grp1** specification
4. To aid the a subscript from the input reduces the number of cos in the list.

#### 1. ib970mrn_grp all have keys= "dtv" and "Cleaned_Timestamp" + a different set of key combinations. [a key is the name of data col]
2. The ib970mrn_grp#_dict {key: grp# .........}
3. Each grp# starts with the "dtv" & "Cleaned_Timestamp" cols ready to have more cols added to the list.
4. A filter that shows all the "df_ib970cln_col_nms" that contain str supplied by input functions.
5. Chosen "col names" are copied into the list for that grp#
6. The grp# is stored via pickel and is read from pick either to use or to edit.
7. When editing, if a grp# does not exist the templet with the 2 cols is profide to star a new group.

### Implementation

#### The following readings are not working
184. Subcutaneous Fat
188. Visceral Fat
195. Abdominal Fat
196. V/S Ratio(Visceral Fat Area/Subcutaneous Fat Area ratio)
197. SFA(Subcutaneous Fat Area)
208. 50khz-Ab Impedance
209. 250khz-Ab Impedance


In [384]:
# make list of data_col names that have subsript ie THIS IS AN AID TO FIND THE RIGHT DA_COLS
df_col_nms = df_ib970cln_col_nms
#filter_column_names_interactive(df_col_nms)
# filter_column_names_interactive(df_col_nms)
# link for ECM&BCM   https://copilot.microsoft.com/shares/J9Fbyou7dZXqDwmDD78S7

In [385]:
# Call the consolidated DEF to build "bi970grp1" TO SEE IF EDITS ARE NEEDED
ib970mrn_grp1_nms = [
    "dtv",
    "Cleaned_Timestamp",
    "15. Weight",
    "226. TBW/WT",
    "217. SMM/WT",
    "142. TBW/FFM",
    "38. SMM (Skeletal Muscle Mass)",
    "33. DLM (Dry Lean Mass)",
    "27. Protein",
    "34. BFM (Body Fat Mass)",
    "185. Arms/Legs Fat",
    "134. VFA (Visceral Fat Area)",
    "V/S Ratio(Visceral Fat Area/Subcutaneous Fat Area ratio)",
    "27. Protein",
    "132. BMR (Basal Metabolic Rate)",
    "135. BCM (Body Cell Mass)",
    "225. ECM/BCM",
    "139. BMC (Bone Mineral Content)",
    "30. Minerals",
    "21. ICW (Intracellular Water)",
    "89. ICW of Trunk",
    "95. ICW of Left Leg",
    "92. ICW of Right Leg",
    "24. ECW (Extracellular Water)",
    "104. ECW of Trunk",
    "110. ECW of Left Leg",
    "113. ECW/TBW",
    "116. ECW/TBW of Trunk",
    "118. ECW/TBW of Left Leg",
    "132. BMR (Basal Metabolic Rate)",
    "145. 5kHz-RA Phase Angle",
    "146. 5kHz-LA Phase Angle",
    "147. 5kHz-TR Phase Angle",
    "148. 5kHz-RL Phase Angle",
    "149. 5kHz-LL Phase Angle",
    "150. 50kHz-RA Phase Angle",
    "151. 50kHz-LA Phase Angle",
    "152. 50kHz-TR Phase Angle",
    "153. 50kHz-RL Phase Angle",
    "154. 50kHz-LL Phase Angle",
    "155. 250kHz-RA Phase Angle",
    "156. 250kHz-LA Phase Angle",
    "157. 250kHz-TR Phase Angle",
    "158. 250kHz-RL Phase Angle",
    "159. 250kHz-LL Phase Angle",
    "160. 50kHz-Whole Body Phase Angle",
    "244. 50kHz-Whole Body Phase Angle_Z score",
    ]

In [386]:
# for verification
# 
ib970mrn_grp1_nms

['dtv',
 'Cleaned_Timestamp',
 '15. Weight',
 '226. TBW/WT',
 '217. SMM/WT',
 '142. TBW/FFM',
 '38. SMM (Skeletal Muscle Mass)',
 '33. DLM (Dry Lean Mass)',
 '27. Protein',
 '34. BFM (Body Fat Mass)',
 '185. Arms/Legs Fat',
 '134. VFA (Visceral Fat Area)',
 'V/S Ratio(Visceral Fat Area/Subcutaneous Fat Area ratio)',
 '27. Protein',
 '132. BMR (Basal Metabolic Rate)',
 '135. BCM (Body Cell Mass)',
 '225. ECM/BCM',
 '139. BMC (Bone Mineral Content)',
 '30. Minerals',
 '21. ICW (Intracellular Water)',
 '89. ICW of Trunk',
 '95. ICW of Left Leg',
 '92. ICW of Right Leg',
 '24. ECW (Extracellular Water)',
 '104. ECW of Trunk',
 '110. ECW of Left Leg',
 '113. ECW/TBW',
 '116. ECW/TBW of Trunk',
 '118. ECW/TBW of Left Leg',
 '132. BMR (Basal Metabolic Rate)',
 '145. 5kHz-RA Phase Angle',
 '146. 5kHz-LA Phase Angle',
 '147. 5kHz-TR Phase Angle',
 '148. 5kHz-RL Phase Angle',
 '149. 5kHz-LL Phase Angle',
 '150. 50kHz-RA Phase Angle',
 '151. 50kHz-LA Phase Angle',
 '152. 50kHz-TR Phase Angle',
 '

### Creating  **ib970mrn_grp1** from **ib970cln**

###  Overview

#### Extracts selected columns from a dict, validates timestamps, sorts chronologically,
    and filters by time-of-day window.

    Parameters:
    - source_dict: dict — your working dict (e.g. ib970cln)
    - column_group: list — list of column names to extract
    - timestamp_col: str — name of the timestamp column
    - start_time: datetime.time — lower bound for time-of-day filter
    - end_time: datetime.time — upper bound for time-of-day filter

    Returns:
    - df_filtered: pd.DataFrame — cleaned, sorted, and time-window-filtered DataFrame
    - df_errors: pd.DataFrame — rows with invalid timestamps
    """

###  Steps going from ib970cln to ib970cl_mrn_grp1
  

#### 
    # Step 1: Extract selected columns
    df = pd.DataFrame({col: source_dict[col] for col in column_group if col in source_dict})

    # Step 2: Validate timestamp column
    if timestamp_col not in df.columns:
        raise ValueError(f"Timestamp column '{timestamp_col}' not found in selected group.")

    df[timestamp_col] = pd.to_datetime(df[timestamp_col], errors="coerce")

    # Step 3: Split valid and invalid timestamps
    df_valid = df[df[timestamp_col].notna()].copy()
    df_errors = df[df[timestamp_col].isna()].copy()

    # Step 4: Sort chronologically
    df_valid = df_valid.sort_values(timestamp_col).reset_index(drop=True)

    # Step 5: Extract time component
    df_valid["time_only"] = df_valid[timestamp_col].dt.time

    # Step 6: Filter by time-of-day window
    df_filtered = df_valid[
        (df_valid["time_only"] >= start_time) &
        (df_valid["time_only"] <= end_time)
    ].copy()

    # Step 7: Drop helper column
    df_filtered.drop(columns=["time_only"], inplace=True)

    return df_filtered, df_errors

### Calling functions to implement the evolution from **ib970cln** to  **ib970cl_mrn_grp1** using the **ib970mrn_grp1_nms**
1. def extract_and_filter_by_time_window(**ib970cln**, **ib970mrn_grp1_nms**)
2. from **sys_funcs.py**

In [387]:
#verify 
# ib970cln

In [388]:
# verify 
# 
ib970mrn_grp1_nms

['dtv',
 'Cleaned_Timestamp',
 '15. Weight',
 '226. TBW/WT',
 '217. SMM/WT',
 '142. TBW/FFM',
 '38. SMM (Skeletal Muscle Mass)',
 '33. DLM (Dry Lean Mass)',
 '27. Protein',
 '34. BFM (Body Fat Mass)',
 '185. Arms/Legs Fat',
 '134. VFA (Visceral Fat Area)',
 'V/S Ratio(Visceral Fat Area/Subcutaneous Fat Area ratio)',
 '27. Protein',
 '132. BMR (Basal Metabolic Rate)',
 '135. BCM (Body Cell Mass)',
 '225. ECM/BCM',
 '139. BMC (Bone Mineral Content)',
 '30. Minerals',
 '21. ICW (Intracellular Water)',
 '89. ICW of Trunk',
 '95. ICW of Left Leg',
 '92. ICW of Right Leg',
 '24. ECW (Extracellular Water)',
 '104. ECW of Trunk',
 '110. ECW of Left Leg',
 '113. ECW/TBW',
 '116. ECW/TBW of Trunk',
 '118. ECW/TBW of Left Leg',
 '132. BMR (Basal Metabolic Rate)',
 '145. 5kHz-RA Phase Angle',
 '146. 5kHz-LA Phase Angle',
 '147. 5kHz-TR Phase Angle',
 '148. 5kHz-RL Phase Angle',
 '149. 5kHz-LL Phase Angle',
 '150. 50kHz-RA Phase Angle',
 '151. 50kHz-LA Phase Angle',
 '152. 50kHz-TR Phase Angle',
 '

In [389]:
# This function uses THIS FUNCTION stored in "sys_funcS.py to use the the ib970mrn_grp1_nms to collect the right data_cols from ib970cln for ib970mrn_grp1
#  IT THEN PERFORMS THE 7 STEPS TO RESULT IN       "ib970mrn_grp1 
column_group = ib970mrn_grp1_nms
# no_ no_no_no_no_The following will take morning an eve reading Put # in front below to to do only mnr
# no_no_no_no_no_no_bring   to this line to do only mrn reads column_group = df_ib970cln_col_nms
#
source_dict = ib970cln
# select the times to get mrn or all
# ib970mrn_grp1,df_errors = extract_and_filter_by_time_window(source_dict, column_group, timestamp_col="Cleaned_Timestamp",start_time=time(4, 0), end_time=time(11, 0))  #mrn
ib970mrn_grp1,df_errors = extract_and_filter_by_time_window(source_dict, column_group, timestamp_col="Cleaned_Timestamp",start_time=time(0, 0), end_time=time(23,59))    #all

In [390]:
# verify
#
# ib970mrn_grp1 #["dtv"] #this is the final result "ib970mrn_grp1"`
# ib970cln #["160. 50kHz-Whole Body Phase Angle"]

In [391]:
# verify to see the rejected teststamps
# df_errors

In [392]:
#ib970mrn_grp1["15. Weight"]    
pd.set_option('display.max_rows', None)
ib970mrn_grp1[['dtv', 'Cleaned_Timestamp','15. Weight']]

,dtv,Cleaned_Timestamp,15. Weight
0,45915.0,2025-09-17 22:32:42,260.7
1,45915.0,2025-09-17 22:32:42,260.7
2,45916.0,2025-09-18 07:45:12,260.1
3,45916.0,2025-09-18 07:45:12,260.1
4,45916.0,2025-09-18 08:09:17,260.1
5,45916.0,2025-09-18 08:09:17,260.1
6,45916.0,2025-09-18 08:09:17,260.1
7,45916.0,2025-09-18 08:09:17,260.1
8,45916.0,2025-09-18 22:44:25,259.8
9,45916.0,2025-09-18 22:44:25,259.8


In [393]:
kind_of(ib970mrn_grp1)

'dataframe'

In [394]:
# Save ib970mrn_grp1 to a file
with open("ib970mrn_grp1.pkl", "wb") as f:
    pickle.dump(ib970mrn_grp1, f)
print(" ib970mrn_grp1 is saved to pickle")

 ib970mrn_grp1 is saved to pickle


In [395]:
# verify
# ib970mrn_grp1 

In [396]:
# Save ib970mrn_grp1_nms to a file
with open("ib970mrn_grp1_nms.pkl", "wb") as f:
    pickle.dump(ib970mrn_grp1_nms, f)
print(" ib970mrn_grp1_nms is saved to pickle")

 ib970mrn_grp1_nms is saved to pickle


# The rest are spares and duplicates >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

In [397]:
print("all cells run ip970mrn_grp1 in pickle")

all cells run ip970mrn_grp1 in pickle
